<center><h1> Projeto de Machine Learning </h1><center>
<center><h1>Previsão de Chuvas na Austrália</h1><center>

------------------------------------------------
<center>Eduardo Selber, Henrique Badin e Luca Caruso<center>

## Preparando o ambiente

In [ ]:
#Importando as bibliotecas
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import sweetviz as sv
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
#Carregando o dataset
df = pd.read_csv("weatherAUS.csv")
df.head(5)

In [ ]:
#Gerando um relatorio de analise exploratoria
#report = sv.analyze(df)
#report.show_notebook()

## Data Wrangling
Primeiro vamos formatar os dados da base de dados para prepará-lo para o modelo preditivo

In [ ]:
#Tratamento de valores ausentes: Deletando as linhas com os valores ausentes das colunas "Raintoday" e "RainTomorrow"
df = df.dropna(subset=["RainToday", "RainTomorrow"])

#Deixando apenas as colunas Date, Location, MinTemp, MaxTemp, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm, Temp9am, Temp3pm, RainToday, RainTomorrow
df = df[["Date", "Location", "MinTemp", "MaxTemp", "Humidity9am", "Humidity3pm", "Pressure9am", "Pressure3pm", "Temp9am", "Temp3pm", "RainToday", "RainTomorrow"]]
df = df.dropna()

# #Tratamento dos outliers e features engineering:
#MinTemp
df = df[(df["MinTemp"] >= -10) & (df["MinTemp"] <= 30)]

#MaxTemp
df = df[(df["MaxTemp"] >= 5) & (df["MaxTemp"] <= 40)]

#Humidity9am não possui outliers
#Humidity3pm não possui outliers

#Pressure9am
df = df[(df["Pressure9am"] >= 1000) & (df["Pressure9am"] <= 1035)]

#Pressure3pm
df = df[(df["Pressure3pm"] >= 1000) & (df["Pressure3pm"] <= 1035)]

#Temp9am
df = df[(df["Temp9am"] >= 5) & (df["Temp9am"] <= 30)]

#Temp3pm
df = df[(df["Temp3pm"] >= 8) & (df["Temp3pm"] <= 35)]

#Tratamento de variaveis categoricas 
df["RainToday"] = df["RainToday"].map({"Yes":1, "No":0})
df['RainTomorrow'] = df['RainTomorrow'].replace({'Yes': 1, 'No': 0})

## EDA - Exploratory Data Analysis
Primeiro vamos analisar os dados que são fornecidos pela base de dados

### Colunas do dataset
- **Date** - A data da observação (formato: yyyy-mm-dd)
- **Location** - A localização da estação meteorológica (Nome da cidade - string)
- **MinTemp** - A temperatura mínima em graus Celsius (float)
- **MaxTemp** - A temperatura máxima em graus Celsius  (float)
- **Rainfall** - A quantidade de chuva medida em mm (float)
- **Evaporation** - A taxa de evaporação medida em mm (float)
- **Sunshine** - O número de horas de sol (float)
- **WindGustDir** - A direção da rajada de vento mais forte (string)
- **WindGustSpeed** - A velocidade (em km/h) da rajada de vento mais forte (float)
- **WindDir9am** - A direção do vento às 9am (string)
- **WindDir3pm** - A direção do vento às 3pm (string)
- **WindSpeed9am** - A velocidade do vento às 9am (em km/h) (float)
- **WindSpeed3pm** - A velocidade do vento às 3pm (em km/h) (float)
- **Humidity9am** - A umidade relativa às 9am (em %) (float)
- **Humidity3pm** - A umidade relativa às 3pm (em %) (float)
- **Pressure9am** - A pressão atmosférica reduzida ao nível do mar às 9am (em hpa) (float)
- **Pressure3pm** - A pressão atmosférica reduzida ao nível do mar às 3pm (em hpa) (float)
- **Cloud9am** - A fração de cobertura de nuvens às 9am (em oitavos) (float)
- **Cloud3pm** - A fração de cobertura de nuvens às 3pm (em oitavos) (float)
- **Temp9am** - A temperatura às 9am em graus Celsius (float)
- **Temp3pm** - A temperatura às 3pm em graus Celsius (float)
- **RainToday** - Se choveu (precipitação acima de 1mm) ou não (string)
- **RainTomorrow** - A variável alvo. Se choverá ou não amanhã (string)


In [ ]:
# Verificando as métricas do dataset
df.describe().transpose()

In [ ]:
# Estatísticas descritivas do DataFrame
df.describe()

print("VARIÁVEIS NUMÉRICAS")
# Distribuição das variáveis numéricas
df.hist(bins=30, figsize=(20, 15), layout=(6, 4))
plt.tight_layout()
plt.show()

# print("VARIÁVEIS CATEGÓRICAS")
# # Contagem das variáveis categóricas do DataFrame
# plt.figure(figsize=(20, 15))
# for i, column in enumerate(df.select_dtypes(include='object').columns, 1):
#     plt.subplot(3, 3, i)
#     sns.countplot(data=df, y=column)
#     plt.title(f'{column} Count')
#     plt.xticks(rotation=45)

In [ ]:
df.shape


In [ ]:
df.head()

In [ ]:
#Filtrando o dataset para a cidade de Sydney
df = df[df["Location"] == "Sydney"]

#Ordenando o dataset pela coluna Date
df = df.sort_values("Date")

In [ ]:
#Dividindo o dataset em train test e validação
#Treinamento: 60%
#Teste: 20%
#Validação: 20%

#quantidade de linhas do dataset
n_lines = df.shape[0]

#separando o dataset em treinamento, teste e validação
train = df[0:int(n_lines*0.6)]
validation = df[int(n_lines*0.6):int(n_lines*0.8)]
test = df[int(n_lines*0.8):]


In [ ]:
def make_x_y(df):
    df = df.drop(["Date", "Location"], axis=1)
    x = df.drop("RainTomorrow", axis=1).copy()
    y = df["RainTomorrow"].copy()
    return x, y
x_train, y_train = make_x_y(train)
x_validation, y_validation = make_x_y(validation)
x_test, y_test = make_x_y(test)

In [ ]:
def make_windows(x, y, columns_orig, window_size):
    x_window = []
    y_window = []
    for i in range(len(x)-window_size-1):
        x_window.append(x[i:i+window_size].reshape(-1))
        y_window.append(y[i+window_size])

    columns = []
    for i in range(window_size):
        for column in columns_orig:
            columns.append(f"{column}_{i}")
    return np.array(x_window), np.array(y_window), columns

window = 7
columns_orig = train.columns[2:-1]
x_train, y_train, columns = make_windows(x_train.values, y_train.values, columns_orig, window)
x_validation, y_validation, _ = make_windows(x_validation.values, y_validation.values, columns_orig, window)

In [ ]:
pd.Series(y_validation).value_counts(normalize=True)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Função para construir o modelo Sequential
def build_sequential_model():
    model = Sequential()
    model.add(Dense(64, input_dim=x_train.shape[1], activation="relu"))  # Camada oculta
    model.add(Dropout(0.3))  # Regularização
    model.add(Dense(32, activation="relu"))  # Outra camada oculta
    model.add(Dense(1, activation="sigmoid"))  # Saída binária
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Modelo Sequential como KerasClassifier
keras_model = KerasClassifier(build_fn=build_sequential_model, epochs=10, batch_size=32, verbose=0)

# Reamostragem da classe minoritária
ros = RandomOverSampler(sampling_strategy="minority")
x_resampled, y_resampled = ros.fit_resample(x_train, y_train)

# Adicionando modelos ao dicionário
modelos = {
    "RandomForest_balanced": RandomForestClassifier(n_estimators=1000, max_depth=10, class_weight="balanced"),
    "GradientBoosting_balanced": GradientBoostingClassifier(n_estimators=100),
    "LogisticRegression_balanced": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "SVC_balanced": SVC(kernel="linear", class_weight="balanced"),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Sequential_Keras": keras_model
}

# Treinar e avaliar cada modelo
for nome, modelo in modelos.items():
    print(f"\nModelo: {nome}")
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", modelo)
    ])
    
    pipeline.fit(x_resampled, y_resampled)
    y_pred_val = pipeline.predict(x_validation)
    
    # Métricas
    print("Matriz de Confusão:")
    print(confusion_matrix(y_validation, y_pred_val))
    print("\nRelatório de Classificação:")
    print(classification_report(y_validation, y_pred_val))
    print("\nAcurácia:", accuracy_score(y_validation, y_pred_val))


In [ ]:
y_pred_val

In [ ]:
import matplotlib.pyplot as plt

# Contagem de corretos e incorretos
certo = 0
total = 0
for i in range(len(y_pred_val)):
    print(y_validation[i], y_pred_val[i])
    if y_validation[i] == y_pred_val[i]:
        certo += 1
    total += 1

errado = total - certo

# Dados para o gráfico
labels = ['Correto', 'Incorreto']
values = [certo, errado]

# Criando o gráfico de barras
plt.bar(labels, values, color=['green', 'red'])
plt.xlabel('Previsões')
plt.ylabel('Quantidade')
plt.title('Quantidade de Previsões Corretas e Incorretas')
plt.show()
print(f'Acurácia: {certo/total*100}%')